# Create some features and labels

WARNING: Anyone is free to use this, but it does override the safe BIOS settings and could absolutely damage your laptop!

I would like to see if I can use machine learning to improve upon a heuristics based fan controllor for my Thinkpad P14s gen 2 intel edition laptop. When I first installed Ubuntu I noticed two very disturbing things. First, the fan consistently runs at a high speed, even when the machine is at idle. This made it unpleasant to use without noise canceling headphones. Second, when the CPU or GPU are used heavily for 10 minutes or so the computer became very slow and unresponsive to input. It turnes out the first is caused by a very aggressive 'auto' fan mode, which keeps the fan at a minimum of 2785 rpm. The second is caused by the bios putting the CPU in an ultro low power mode if it it hits 50C for more than 10 minutes. Ubuntu has a throttled deamon that is supposed to prevent the second state. This deamon was developed for Thinkpads, but it didn't run on my laptop. I found out later I could edit the throttled.service so it would run and manually set the fan to level 1 with the thinkpad_acpi and get a decent laptop without any of this tinkering. If you are horrified by your Lenovo's performance or noise level, this is probably all you need to do. But my heuristic controller is already better for me and I'd like to use this to try evolving a heuristic into increasingly sophisticated ML models. I'm going to overengineer the hell out of this for fun and absolutely no profit.

So, I've created a heuristic based fan and power budget controller. Using pyarrow, I'm logging to parquet various features along with a label of fan speed level from the heuristic fan controller.

I'm using intel_pstate driver to optimize for performance. It has two states, powersave and performance, for now we're always leaving it in powersave mode. In this mode the driver attempts to "race to idle". That is it will run at full speed within the current power budget and in an attempt to finish all work and then suspend as many CPUs as possible.

The power budget is set via a separete heuristic. Basically if the fan is at speed 1 or below we give full power, if it is at speed 2 we give it a bit less power, and if it is running at speeds 3 to 7 we put the CPU in low power mode at about 12 watts. If we never set the wattage the BIOS would force the machine into a ultra low power mode after the temperature has been above 49.5C for about ten minutes. In this state the power budget is only 5 watts and the machine will only run at 400Mhz. Once in this mode it never exits unless you set the power budget explicitly. At some point I would like to control this with the same ML based controller, but I want to start more simply. 

I'm using a simple reward function as my metric.

Initially I'll treat this as a supervised learning problem so can investigate what kind of memory will be most useful. i.e. RNN or Attention. For supervised learning I'll definitely need to reweigh or resample the data, as most of the examples are from when the machine is at rest.

Then I want to try an RL approach using the reward function directly.

At least to start the sensors read of the temperatures is the most CPU intesive thing we're doing. I'm occasionally running under cProfile. But for now I'm not to worried about this as I'm mostly running off mains. I will probably need to develop a separate heuristic and reward function for battery use and then I'll revisit running the action function every 200 ms as we do now or querying the temperature sensors more efficiently (there is also  /proc/acpi/ibm/thermal provided by the thinkpad_acpi driver).

Known dependencies:
apt: lm-sensors
python: see requirements.txt
kernel modules: thinkpad_acpi, intel_rapl_msr
kernel params:  processor.ignore_ppc=1 intel_pstate=enable